In [ ]:
# Bibliotecas Importadas
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pnd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
listas = ['month_2.csv', 'month_3.csv', 'month_4.csv', 'month_5.csv', 'month_6.csv']
df = []
for arquivo in listas:
    df += [pnd.read_csv(arquivo)]
#Concatena todos os dataframes em um único dataframe chamado df
df = pnd.concat(df)
#Chama o dataframe contido na variável chamada df
dadosCadastrais = pnd.read_csv('informacao_cadastral.csv')
usuariosUnicos = dadosCadastrais[dadosCadastrais.situacao == 'CONSUMINDO GÁS']['clientCode'].unique() 
#Organiza os dados dos usuários filtrados pela data
mesFiltrado = df[df['clientCode'].isin(usuariosUnicos)].sort_values(by='datetime') 
#Filtra meterSN diferente de '>N<A'
df = mesFiltrado[mesFiltrado['meterSN'] != '>N<A']
#Garante que todas as linhas com gain nulo sejam preenchidas com 1. Não é garantido que é o valor correto, mas é o melhor que podemos fazer
df['gain'].fillna(1, inplace=True)
#Corrige os pulsos para m²
df['pulseCount'] = df['pulseCount'] * df['gain']
#Cria a variação do pulseCount como uma coluna nova, calculando por grupo a diferença
df['datetime'] = pnd.to_datetime(df['datetime'])
df['dateTimeSegundos'] = df['datetime'].astype(np.int64) // 10**9
df['diffDateTime'] = df.groupby(['clientCode', 'meterSN']).dateTimeSegundos.diff()
df['diffPulseCount'] = df.groupby(['clientCode', 'meterSN']).pulseCount.diff()
df['diffPulseCountTempo'] = df['diffPulseCount'] / df['diffDateTime']
#Preenche os valores nulos (iniciais) com 0
df['diffDateTime'].fillna(0, inplace=True) 
df['diffPulseCount'].fillna(0, inplace=True)
df['diffPulseCountTempo'].fillna(0, inplace=True)
#Reseta o index
df.reset_index(drop=True, inplace=True)
#Seleciona as colunas que serão usadas
df = df[['clientCode', 'meterSN', "pulseCount", 'diffPulseCount','datetime', 'diffDateTime', 'diffPulseCountTempo', 'dateTimeSegundos']]
#Calcula a média e o desvio padrão do diffPulseCount por cliente
df['mediaCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('mean')
df['desvioPadraoCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCount.transform('std')
df['diffDateTime'].describe()

In [ ]:
dfCliente = df[df['clientCode'].str.contains('b484c8b5ec41d7d82abdfda82fd5f14c076db223dbd2bc')]
dfCliente.sort_values(by='datetime', inplace=True)
dfCliente.reset_index(drop=True, inplace=True)
dfCliente

In [ ]:
df.sort_values(by='diffDateTime', inplace=True, ascending=False)
df.reset_index(drop=True, inplace=True)
df[df['diffDateTime'] < 90000]

In [ ]:
# Filtrando e ordenando os dados por cliente e tempo
dfCliente = df[df['clientCode'].str.contains('9479bbab64308ce0789119e6dc874805d07f386dd45c98')]
dfCliente.sort_values(by='datetime', inplace=True)

# Plotando o gráfico de diffDateTime ao longo do tempo
plt.figure(figsize=(12, 6))
plt.plot(dfCliente['datetime'], dfCliente['diffDateTime'], marker='o', linestyle='-', color='r')
plt.title('Variação do diffDateTime ao longo do tempo')
plt.xlabel('Data e Hora')
plt.ylabel('Diff DateTime (segundos)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

In [ ]:
# Filtrando e ordenando os dados por cliente e tempo
dfCliente = df[df['clientCode'].str.contains('b4e5384402e135d94a96529f98c6eda68017dd25628243')]
dfCliente.sort_values(by='datetime', inplace=True)

# Plotando o gráfico de diffDateTime ao longo do tempo
plt.figure(figsize=(12, 6))
plt.plot(dfCliente['datetime'], dfCliente['diffPulseCountTempo'], marker='o', linestyle='-', color='r')
plt.title('Variação do diffDateTime ao longo do tempo')
plt.xlabel('Data e Hora')
plt.ylabel('Diff DateTime (segundos)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

In [ ]:
dfCliente.sort_values(by='diffDateTime', inplace=True, ascending=False)
dfCliente.reset_index(drop=True, inplace=True)
dfCliente[dfCliente['diffDateTime'] < 90000]

In [ ]:
# Filtrando e ordenando os dados por cliente e tempo
dfCliente = df[df['clientCode'].str.contains('41462ff695d437dfe98a3379e01b48773da62c8f418889')]
dfCliente.sort_values(by='datetime', inplace=True)

# Plotando o gráfico de diffDateTime ao longo do tempo
plt.figure(figsize=(12, 6))
plt.plot(dfCliente['datetime'], dfCliente['diffDateTime'], marker='o', linestyle='-', color='r')
plt.title('Variação do diffDateTime ao longo do tempo')
plt.xlabel('Data e Hora')
plt.ylabel('Diff DateTime (segundos)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

In [ ]:
df['mediaPCTCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCountTempo.transform('mean')
df['desvioPadraoPCTCliente'] = df.groupby(['clientCode', 'meterSN']).diffPulseCountTempo.transform('std')

df['tipo'] = 0
df.loc[df['diffPulseCountTempo'] > df['mediaPCTCliente'] + 2 * df['desvioPadraoPCTCliente'], 'tipo'] = 2
df.loc[df['diffPulseCountTempo'] > df['mediaPCTCliente'] + 3 * df['desvioPadraoPCTCliente'], 'tipo'] = 3
df.loc[df['diffPulseCountTempo'] < 0, 'tipo'] = 1
df.loc[(df['pulseCount'] == 0) & (df['diffPulseCountTempo'] < 0), 'tipo'] = 2
#df.tipo.value_counts()
print(df.tipo.value_counts())
df